In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import os

import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import pygwalker as pyg

In [ ]:
DEFAULT_INPUT_FILE = 'example_data.csv'

In [ ]:
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR')
OUTPUT_DATA_DIR = os.environ.get('OUTPUT_DATA_DIR')
assert INPUT_DATA_DIR is not None
assert OUTPUT_DATA_DIR is not None

In [ ]:
def load_data(file_name: str=DEFAULT_INPUT_FILE):
    f = os.path.join(INPUT_DATA_DIR, file_name)
    return pd.read_csv(f)

In [ ]:
# main
df = load_data()

In [ ]:
sns.pairplot(df)

In [ ]:
# put index to Details field
pyg.walk(df.reset_index())

In [ ]:
target_col = df.columns[0]
feature_cols = df.drop(target_col, axis=1).columns.tolist()

# featureから2つ選んでリストにする
feature_col_pairs = [[feature_cols[i], feature_cols[j]] for i in range(len(feature_cols)) for j in range(i+1, len(feature_cols))]

In [ ]:
for feature_col_pair in feature_col_pairs:
    X = df[feature_col_pair]
    y = df[target_col]

    model = DecisionTreeRegressor()
    model.fit(X, y)
    y_pred = model.predict(X)

    df_pred = df[feature_col_pair].copy()
    df_pred[target_col] = y
    df_pred[f'{target_col}_pred'] = y_pred

    # assessment
    accuracy = model.score(X, y)
    print(accuracy)

    display(df_pred.sort_values(feature_col_pair))